In [11]:
np.random.uniform(20,40, (3,2))

array([[ 20.17604371,  28.85722059],
       [ 20.50391418,  34.1493168 ],
       [ 23.53926302,  26.73893768]])

# Sensitivity analysis in the finite approximation to the IBP
These results should be the same as in the LRVB notebook 

The difference being that here, functions are written using the VB library

In [1]:
import autograd.numpy as np
import autograd.scipy as sp
from autograd.scipy import special
from autograd import grad, hessian, hessian_vector_product, hessian, jacobian
import matplotlib.pyplot as plt
%matplotlib inline  

from copy import deepcopy

from scipy import optimize

import time

import valez_finite_VI_lib as vi
import LRVB_lib as lrvb
import generic_optimization_lib as packing

import sys
sys.path.append('../../LinearResponseVariationalBayes.py')
from VariationalBayes.ParameterDictionary import ModelParamsDict
from VariationalBayes.Parameters import ScalarParam


### Generate data

In [2]:
np.random.seed(12321)

alpha = 10 # IBP parameter

num_samples = 50 # sample size
D = 2 # dimension

sigma_a = 3.0 ** 2

sigma_eps = 1.0 ** 2 # variance of noise

k_inf = 3 # take to be large for a good approximation to the IBP

# generate data
pi, Z, A, X = vi.generate_data(num_samples, D, k_inf, sigma_a, sigma_eps, alpha)
   

### Initialize variational parameters

In [3]:
k_approx = k_inf # variational truncation

tau_init, nu_init, phi_mu_init, phi_var_init = \
    vi.initialize_parameters(num_samples, D, k_approx)

# define vb model -- vb_model is a class defined using the VB library
vb_model = vi.set_ibp_vb_model(num_samples, D, k_approx)

# initialize parameters
vb_model['phi'].set_vector(np.hstack([np.ravel(phi_mu_init.T), phi_var_init]))
vb_model['pi'].set_vector(np.ravel(tau_init))
vb_model['nu'].set_vector(np.ravel(nu_init))

# consolidate hyper parameters
hyper_params = ModelParamsDict('hyper_params')
hyper_params.push_param(ScalarParam('alpha', lb = 0.0))
hyper_params.push_param(ScalarParam('var_a', lb = 0.0))
hyper_params.push_param(ScalarParam('var_eps', lb = 0.0))

hyper_params['alpha'].set(alpha)
hyper_params['var_a'].set(sigma_a)
hyper_params['var_eps'].set(sigma_eps)

data_set = lrvb.DataSetII(X, vb_model, hyper_params)

In [4]:
import functional_perturbation_lib as fun_pert
from scipy import integrate
from scipy.stats import beta
import scipy as osp

print(fun_pert.compute_elbo_perturbed(X, vb_model, hyper_params))
print(vi.compute_elboII(X, vb_model, hyper_params))

[[ 0.72856094  1.6083226 ]
 [ 1.08473534  1.83491632]
 [ 1.38764972  1.33819602]]
10
3
1000000
[-2.73385318 -1.89623186 -0.85350011]
beta_lh -5.48358515324
e_log_lik -169.821755891
entropy:  85.8272964727
-83.9944594184
beta_lh -5.67536251536
e_log_lik -170.013533253
entropy 85.8272964727
-84.1862367805


In [5]:
vb_model['phi'].info.get()

array([ 1.,  1.,  1.])

In [7]:
tau = vb_model['pi'].alpha.get()
print(tau)
test = fun_pert.compute_e_pi_prior_perturbed(tau, alpha, k_approx, n_grid = 10**6)
print(test)

for k in range(k_approx): 
    integrand = lambda x : beta.pdf(x, tau[k,0], tau[k,1]) * beta.logpdf(x, alpha/k_approx, 1) 
    print(integrate.quad(integrand, 0,1))

(alpha/k_approx - 1) * (osp.special.digamma(tau[:,0]) - osp.special.digamma(tau[:,0] + tau[:,1])) -\
            osp.special.betaln(alpha/k_approx, 1)

[[ 0.72856094  1.6083226 ]
 [ 1.08473534  1.83491632]
 [ 1.38764972  1.33819602]]
[-2.73385318 -1.89623186 -0.85350011]
(-2.9061133280453926, 5.706782602032945e-09)
(-1.914287767670875, 6.532668184178192e-09)
(-0.8549614196382784, 1.1903089625064922e-10)


array([-2.90611333, -1.91428777, -0.85496142])

In [8]:
test * 2

array([-5.46770636, -3.79246372, -1.70700022])

In [6]:
np.sum([-2.90611333, -1.91428777, -0.85496142])

-5.6753625200000002

In [ ]:
x = 0.5

print(beta.logpdf(x, alpha/k_approx, 1) )

print((alpha/k_approx - 1) * np.log(x) - osp.special.betaln(alpha/k_approx, 1))

In [ ]:
# just checking some things about vb_model

print('vb_model[phi] is an array of normals; \neach component has a mean, and each row has a common variance')
print(vb_model['phi'].mean.get())
print(vb_model['phi'].info.get(), '\n')
assert np.all(phi_mu_init.T == vb_model['phi'].e())

print('vb_model[pi] is an array where each row is dirichlet \nwe can get its \
dirchlet parameter tau, and we can get its mean')
print(vb_model['pi'].alpha.get())
print(vb_model['pi'].e(), '\n')
assert np.all(tau_init == vb_model['pi'].alpha.get())

print('vb_model[pi] is an ArrayParam class where each element is constrained to be between 0 and 1')
print(vb_model['nu'].get())
assert np.all(nu_init == vb_model['nu'].get())

### plot the data

In [ ]:
col = 0
plt.figure()
plt.hist(data_set.x[:, col], bins=100);

col1 = 0
col2 = 1
plt.figure()
plt.plot(data_set.x[:, col1], data_set.x[:, col2], 'k.');

### Run CAVI

In [ ]:
# Parameters approximating the true distribution

tau_true = np.zeros_like(tau_init)
tau_true_scale = 15.
tau_true[:, 0] = deepcopy(pi) * tau_true_scale
tau_true[:, 1] = tau_true_scale

nu_true = np.zeros_like(nu_init)
nu_true[Z == 1] = 0.999
nu_true[Z == 0] = 0.001

phi_mu_true = np.zeros_like(phi_mu_init)
phi_mu_true[:] = A.transpose()
phi_var_true = np.zeros_like(phi_var_init)
phi_var_true[:] = 0.01

params_true = packing.pack_params(deepcopy(tau_true), deepcopy(phi_mu_true),
                                  deepcopy(phi_var_true), deepcopy(nu_true))

In [ ]:
true_init = False
if true_init:
    vb_model['phi'].set_vector(np.hstack([np.ravel(phi_mu_true.T), phi_var_true]))
    vb_model['pi'].set_vector(np.ravel(tau_true))
    vb_model['nu'].set_vector(np.ravel(nu_true))
    tau, phi_mu, phi_var, nu = data_set.unpack_params(vb_model)
else:
    # the random initialization was done above
    tau, phi_mu, phi_var, nu = data_set.unpack_params(vb_model)
    

    tau, nu, phi_mu, phi_var = data_set.run_cavi(tau, nu, phi_mu, phi_var, max_iter=100, tol=1e-6)

cavi_tau = deepcopy(tau)
cavi_phi_mu = deepcopy(phi_mu)
cavi_phi_var = deepcopy(phi_var)
cavi_nu = deepcopy(nu)

In [ ]:
# CAVI can return nu values that are too close to 0 or 1 for the encoding.
nu_tol = 1e-8
cavi_nu_trim = deepcopy(cavi_nu)
cavi_nu_trim[cavi_nu_trim < nu_tol] = nu_tol
cavi_nu_trim[cavi_nu_trim > 1 - nu_tol] = 1 - nu_tol

cavi_params = packing.pack_params(cavi_tau, cavi_phi_mu, cavi_phi_var, cavi_nu_trim)
print(np.all(np.isfinite(cavi_params)))

vb_model.set_free(cavi_params)
cavi_resid = data_set.x - data_set.get_prediction(vb_model)

In [ ]:
data_set.trace.reset()
vb_opt = data_set.run_newton_tr(cavi_params, maxiter=50, gtol=1e-2)
vb_model.set_free(vb_opt.x)

In [ ]:
tau_final = vb_model['pi'].alpha.get()

from scipy.stats import beta

x = np.linspace(.001, 0.999, 100)
y = beta.pdf(x, tau_final[0,0], tau_final[0,1])

plt.plot(x,y)

In [ ]:
tau_final

In [ ]:
print ('CAVI:')
print (cavi_phi_mu.transpose())

print ('Full TR:')
print (vb_model['phi'].mean.get())

print ('Truth:')
print (A)


In [ ]:
tr_resid = data_set.x - data_set.get_prediction(vb_model)
true_resid = data_set.x - np.matmul(Z, A)

plt.figure()
col = 0
plt.plot(data_set.x[:, col], tr_resid[:, col], '.b')
plt.plot(data_set.x[:, col], cavi_resid[:, col], '.r')
plt.plot(data_set.x[:, col], true_resid[:, col], '.g')
plt.plot(data_set.x[:, col], np.full_like(data_set.x[:, col], 0.), 'k')

print('Cavi residuals: {}    Trust residuals: {}      True residuals: {}'.format(
       np.sum(np.abs(cavi_resid)), np.sum(np.abs(tr_resid)), np.sum(np.abs(true_resid))))


In [ ]:
# Compute prior sensitivity, eq. 18 in the paper
moment_sensitivity = data_set.local_prior_sensitivity()

#print(moment_sensitivity)
#np.shape(moment_sensitivity)

In [ ]:
# The third column is sigma_eps.
sigma_eps_col = 2
e_log_pi_sigma_eps_sens, e_mu_sigma_eps_sens = \
    packing.unpack_moments(moment_sensitivity[:, sigma_eps_col], k_approx, D)
    
print('Sensitivity of e_log_pi to sigma_eps:')
print(e_log_pi_sigma_eps_sens)

print('Sensitivity of e_mu to sigma_eps:')
print(e_mu_sigma_eps_sens)

In [ ]:
"""# Perturb and re-rerun to check the sensitivity.

epsilon = 1e-1
data_set_perturb = lrvb.DataSet(X, k_approx, alpha, sigma_eps + epsilon, sigma_a)
data_set_perturb.trace.print_every = 1
vb_opt_perturb = data_set_perturb.run_newton_tr(vb_opt.x)"""

In [ ]:
"""e_log_pi, e_mu = data_set.get_moments(tr_params)
e_log_pi_perturb, e_mu_perturb = data_set.get_moments(vb_opt_perturb.x)

print('Measured sensitivity of e_mu to sigma_eps:')
print((e_mu_perturb - e_mu) / epsilon)

print('Sensitivity of e_mu to sigma_eps:')
print(e_mu_sigma_eps_sens)

print('Measured sensitivity of e_log_pi to sigma_eps:')
print((e_log_pi_perturb - e_log_pi) / epsilon)

print('Sensitivity of e_log_pi to sigma_eps:')
print(e_log_pi_sigma_eps_sens)
"""

In [ ]:
data_set.influence_function_pi(0.8, 1)

In [ ]:
# plot influence function

n_ticks = 100
y = np.zeros((n_ticks, k_approx))

post_pi = 0 # which pi you're looking at in the posterior

theta = np.linspace(0.001,0.999,n_ticks)

for k in range(k_approx):  
    for i in range(n_ticks): 
        y[i, k] = data_set.influence_function_pi(theta[i], k)[post_pi]
        
    plt.plot(theta, y[:,k])
    plt.xlabel('theta')
    plt.title('Marginal Influence on E[log pi' + str(post_pi) + '],\nwith prior perturbation on pi_' + str(k))
    plt.show()

plt.plot(theta, np.sum(y,1))
plt.xlabel('theta')
plt.title('Total Influence on E[log pi' + str(post_pi) + ']')
plt.show()


In [ ]:
vi.compute_elboII(X, vb_model, hyper_params)